Bu veri seti Covid-19 olarak ortaya çıkan virüs için dünya genelinde ‘Covid Pozitif Hasta Sayısı’ , ‘İyileşen Hasta Sayısı’ , ‘Hayatını Kaybeden Hasta Sayısı’ değerlerini içermektedir.

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session


-> Seçilen veri seti import edildi ve içindeki veriler okutuldu.

In [ ]:
import pandas as pd
covid=pd.read_csv("../input/novel-corona-virus-2019-dataset/covid_19_data.csv")
covid.head()


In [ ]:
covid.shape

-> Veri setindeki satır sayısı,veri tipi ve değişken isimleri öğrenilerek dataset hakkında genel bilgiye sahip olundu.

In [ ]:
# Satir Sayisi
print("Satır Sayısı:\n",covid.shape[0:])

# Sutun Adlari
print("Sütun Adlari:\n",covid.columns.tolist())

# Veri Tipleri
print("Veri Tipleri:\n",covid.dtypes)

->  Dataset üzerinde analiz , modelleme yapabilmek için eksik veri sayısı kontrolü yapıldı. 

In [ ]:
# Eksik veri sayıları ve veri setindeki oranları 
import matplotlib.pyplot as plt
import seaborn as sns
pd.concat([covid.isnull().sum(), 100 * covid.isnull().sum()/len(covid)], 
              axis=1).rename(columns={0:'Missing Records', 1:'Percentage (%)'})



-> Yeni oluşturacağım datasette enlem ve boylam değerlerini de alabilmek için diğer kullanacağım 'COVID19_open_line_list.csv' dosyası import edildi ve okutuldu.

In [ ]:
import pandas as pd
covid1=pd.read_csv("../input/novel-corona-virus-2019-dataset/COVID19_open_line_list.csv")
covid1.head()

In [ ]:
covid1.shape

-> Amerika için "Deaths, Recovered ve Confirmed" arasındaki ilişkiyi gözlemleyebilmek adına korelasyon grafiği oluşturuldu.

In [ ]:
#Amerika için korelasyon grafiği

us=covid.copy()
y = (us['Country/Region'] == 'US').astype(int)
fields = list(us.columns[1:])  # everything except "country name"
correlations = us[fields].corrwith(y)
correlations.sort_values(inplace=True)
correlations

In [ ]:
ax = correlations.plot(kind='bar')
ax.set(ylim=[-1, 1], ylabel='US Correlation');

-> Veri setleri içerisinden belirli alanlar seçilerek yeni bir veriseti oluşturuldu. Sonra dataframedeki sürekli değişkenler için describe metodu ile "count,mean ,min ,max" değerleri öğrenildi.

In [ ]:
df1=pd.Series(covid['Country/Region'],name="Country")
df2=pd.Series(covid['Last Update'],name="Last")
df3=pd.Series(covid['Confirmed'],name="Confirmed")
df4=pd.Series(covid['Deaths'],name="Deaths")
df5=pd.Series(covid['Recovered'],name="Recovered")
df6=pd.Series(covid1['latitude'],name="lat")
df7=pd.Series(covid1['longitude'],name="long")

df_covid=pd.concat([df1, df2,df3, df4,df5,df6,df7], axis=1)
df_covid.describe().T

-> Aykırı değer gözlemi yapabilmek adına sürekli değişkenler için boxplotlar oluşturuldu. Aykırı değer analizi ile değişkenler içerisindeki değerlerin ortalama ile mi seyrettiği yoksa büyük farklılıkların mı olduğu sonucuna varırız.

In [ ]:
plt.figure()
df_covid.boxplot(column=['Confirmed','Deaths','Recovered'])

fig,axs=plt.subplots(2,2) 
axs[0, 0].boxplot(df_covid['Confirmed'])
axs[0, 0].set_title('Hasta Sayısı')

axs[0, 1].boxplot(df_covid['Recovered'])
axs[0, 1].set_title('İyileşen Hasta Sayısı')

axs[1, 0].boxplot(df_covid['Deaths'])
axs[1, 0].set_title('Hayatını Kaybeden Hasta Sayısı')



-> Dataset içerisinde yer alan ülkeler için bölge bazında değerlendirme yapabilmek adına datasete 'Region' değişkeni eklendi.

In [ ]:
who_region = {}

# African Region 
africa = "Algeria, Angola, Cabo Verde, Eswatini, Sao Tome and Principe, Benin, South Sudan, Western Sahara, Congo (Brazzaville), Congo (Kinshasa), Cote d'Ivoire, Botswana, Burkina Faso, Burundi, Cameroon, Cape Verde, Central African Republic, Chad, Comoros, Ivory Coast, Democratic Republic of the Congo, Equatorial Guinea, Eritrea, Ethiopia, Gabon, Gambia, Ghana, Guinea, Guinea-Bissau, Kenya, Lesotho, Liberia, Madagascar, Malawi, Mali, Mauritania, Mauritius, Mozambique, Namibia, Niger, Nigeria, Republic of the Congo, Rwanda, São Tomé and Príncipe, Senegal, Seychelles, Sierra Leone, Somalia, South Africa, Swaziland, Togo, Uganda, Tanzania, Zambia, Zimbabwe"
africa = [i.strip() for i in africa.split(',')]
for i in africa:
    who_region[i] = 'africa'
    
# Region of the Americas 
america = 'Antigua and Barbuda, Argentina, Bahamas, Barbados, Belize, Bolivia, Brazil, Canada, Chile, Colombia, Costa Rica, Cuba, Dominica, Dominican Republic, Ecuador, El Salvador, Grenada, Guatemala, Guyana, Haiti, Honduras, Jamaica, Mexico, Nicaragua, Panama, Paraguay, Peru, Saint Kitts and Nevis, Saint Lucia, Saint Vincent and the Grenadines, Suriname, Trinidad and Tobago, United States, US, Uruguay, Venezuela'
america = [i.strip() for i in america.split(',')]
for i in america:
    who_region[i] = 'america'

# South-East Asia Region 
asia = 'Bangladesh, Bhutan, North Korea, India, Indonesia, Maldives, Myanmar, Burma, Nepal, Sri Lanka, Thailand, Timor-Leste'
asia = [i.strip() for i in asia.split(',')]
for i in asia:
    who_region[i] = 'asia'
    
# European Region 
euro = 'Albania, Andorra, Greenland, Kosovo, Holy See, Liechtenstein, Armenia, Czechia, Austria, Azerbaijan, Belarus, Belgium, Bosnia and Herzegovina, Bulgaria, Croatia, Cyprus, Czech Republic, Denmark, Estonia, Finland, France, Georgia, Germany, Greece, Hungary, Iceland, Ireland, Israel, Italy, Kazakhstan, Kyrgyzstan, Latvia, Lithuania, Luxembourg, Malta, Monaco, Montenegro, Netherlands, North Macedonia, Norway, Poland, Portugal, Moldova, Romania, Russia, San Marino, Serbia, Slovakia, Slovenia, Spain, Sweden, Switzerland, Tajikistan, Turkey, Turkmenistan, Ukraine, United Kingdom, Uzbekistan'
euro = [i.strip() for i in euro.split(',')]
for i in euro:
    who_region[i] = 'euro'

# Eastern Mediterranean Region 
emro = 'Afghanistan, Bahrain, Djibouti, Egypt, Iran, Iraq, Jordan, Kuwait, Lebanon, Libya, Morocco, Oman, Pakistan, Palestine, West Bank and Gaza, Qatar, Saudi Arabia, Somalia, Sudan, Syria, Tunisia, United Arab Emirates, Yemen'
emro = [i.strip() for i in emro.split(',')]
for i in emro:
    who_region[i] = 'emro'

# Western Pacific Region 
wpro = 'Australia, Brunei, Cambodia, China, Cook Islands, Fiji, Japan, Kiribati, Laos, Malaysia, Marshall Islands, Micronesia, Mongolia, Nauru, New Zealand, Niue, Palau, Papua New Guinea, Philippines, South Korea, Samoa, Singapore, Solomon Islands, Taiwan, Taiwan*, Tonga, Tuvalu, Vanuatu, Vietnam'
wpro = [i.strip() for i in wpro.split(',')]
for i in wpro:
    who_region[i] = 'wpro'



In [ ]:
df_covid['Region'] = df_covid['Country'].map(who_region)
df_covid[df_covid['Region'].isna()]['Region'].unique()

In [ ]:
df_covid['Region'].unique()

In [ ]:
df_covid["Region"].fillna("Other", inplace = True) 

In [ ]:
df_covid.isnull().sum()

-> İçerisinde ülkelerin koordinatlarının da yer aldığı yeni dataframe ile harita üzerinden ülkelerin ‘Covid Pozitif Hasta Sayısı’(Confirmed) , ‘İyileşen Hasta Sayısı’(Recovered) , ‘Hayatını Kaybeden Hasta Sayısı’(Deaths) değerleri görselleştirildi.

In [ ]:
import folium
# Dünya Geneli
temp = df_covid[df_covid['Last'] == max(df_covid['Last'])]
m = folium.Map(location=[0, 0], titles='Dünya Haritası Üzerindeki Değerler',
               min_zoom=1, max_zoom=4, zoom_start=1)

for i in range(0, len(temp)):
    folium.Circle(
        location=[temp.iloc[i]['lat'], temp.iloc[i]['long']],
        color='crimson', fill='crimson',
        tooltip =   '<li><bold>Country : '+str(temp.iloc[i]['Country'])+
                    '<li><bold>Province : '+str(temp.iloc[i]['Region'])+
                    '<li><bold>Confirmed : '+str(temp.iloc[i]['Confirmed'])+
                    '<li><bold>Deaths : '+str(temp.iloc[i]['Deaths']),
        radius=int(temp.iloc[i]['Confirmed'])**0.5).add_to(m)
m


In [ ]:
# Zaman İçerisindeki Değişim
import plotly.express as px
fig = px.choropleth(df_covid, locations="Country", locationmode='country names', color=np.log(df_covid["Confirmed"]), 
                    hover_name="Country", animation_frame=df_covid["Last"],
                    title='Zaman İçerisindeki Değişim', color_continuous_scale=px.colors.sequential.Purp)
fig.update(layout_coloraxis_showscale=False)
fig.show()

-> Oluşturulan "line" plotlar üzerinde 'Region' değişkeni görselleştirildi. Böylelikle bölgelere göre "Test Sonucu Pozitif Olan Hasta Sayısı,İyileşen Hasta Sayısı,Hayatını Kaybeden Hasta Sayısı" gözlemlenmiş oldu.

In [ ]:
import plotly.express as px



fig = px.line(df_covid.sort_values("Confirmed"),
            x='Region', y="Confirmed",
            hover_name="Region",
            hover_data=["Recovered","Deaths","Confirmed"],
            title='COVID-19: Bölgelere Göre Test Sonucu Pozitif Olan Hasta Sayısı',
)
fig.update_xaxes(title_text="Region")
fig.update_yaxes(title_text="Positif Test Sayısı(%)")
fig.show()
fig = px.line(df_covid.sort_values("Recovered"),
            x='Region', y="Recovered",
            hover_name="Region",
            hover_data=["Confirmed","Deaths","Recovered"],
            title='COVID-19: Bölgelere Göre İyileşen Hasta Sayısı ',
)
fig.update_xaxes(title_text="Region")
fig.update_yaxes(title_text="İyileşen Hasta Sayısı")
fig.show()
fig = px.line(df_covid.sort_values("Deaths"),
            x='Region', y="Deaths",
            hover_name="Region",
            hover_data=["Confirmed","Recovered","Deaths"],
            title='COVID-19:  Bölgelere Göre Hayatını Kaybeden Hasta Sayısı ',
)
fig.update_xaxes(title_text="Region")
fig.update_yaxes(title_text="Hayatını Kaybeden Hasta Sayısı")
fig.show()

-> Oluşturulan dataframe için 'Deaths ,Recovered ,Confirmed ' değişkenleri üzerinden 'Recovered'(İyileşen hasta sayısı) üzerine tahminleme yapıldı.

Veri modellemeden önce normalize edildi.Bunu yaparken MinMaxScaler kullanıldı.

Bu yöntemde, bir grup verinin içerisindeki en büyük ve en küçük değerler ele alınır. Diğer bütün veriler, bu değerlere göre normalleştirilir.

Yapılan başarı kıyaslamaları sonuçlarına bakıldığında en iyi sonuçlar Lineer Regresyon, Linear Discriminant Analysis ve Decision Tree Classifier algoritmaları ile elde edilmiştir.


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

X = df_covid.iloc[:,2:5]
y = df_covid['Recovered']

mms = MinMaxScaler()
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.3, random_state=0)
X_train = mms.fit_transform(X_train) 
X_test= mms.fit_transform(X_test)

print("Dataframe boyutu: ",df_covid.shape)
print("Eğitim verisi boyutu: ",X_train.shape, y_train.shape)
print("Test verisi boyutu: ",X_test.shape,y_test.shape)

In [ ]:
# type error için target typesı "Label Encoder" ile  multiclassa çevirdim.(Target=Y_train)
from sklearn import preprocessing
from sklearn import utils

lab_enc = preprocessing.LabelEncoder()
encoded = lab_enc.fit_transform(y)
print(utils.multiclass.type_of_target(y))
print(utils.multiclass.type_of_target(y_train.astype('int')))
print(utils.multiclass.type_of_target(encoded))

lab_enc = preprocessing.LabelEncoder()
Y_train = lab_enc.fit_transform(y_train)

In [ ]:
import numpy as np
from sklearn    import metrics, svm
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn import  linear_model

-> Her bir modelin doğruluk değeri ,sınıflandırma raporu , karışıklık matrisi ve MSE(Ortalama Kare Hata Regresyon Oranı) değerlerini hesaplamak için import edildi.

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import mean_squared_error

-> Yapılan tahminleme sonucunda alınan tahmin değerleri ve doğruluk değerleri ilişkisi gösterildi.

In [ ]:
# Lineer Regresyon
print("\nLineer Regresyon")
lm = linear_model.LinearRegression()
model = lm.fit(X_train, Y_train)
y_true1 , y_pred1 =y_test,lm.predict(X_test)
print("\nTahmin değerleri: ",y_pred1)
plt.scatter(y_true1, y_pred1,c='blue')
plt.scatter(y_true1, y_test,c='pink')
plt.xlabel("True Values")
plt.ylabel("Predictions")

In [ ]:
#Lineer Regresyon
#predictions multiclass olduğundan y_validation da multiclassa dönüştürüldü
encoded_v = lab_enc.fit_transform(y_true1)
utils.multiclass.type_of_target(y_true1.astype('int'))
ypred1= lab_enc.fit_transform(y_pred1)
utils.multiclass.type_of_target(ypred1.astype('int'))
conf=confusion_matrix(encoded_v, ypred1)
print("\nConfusion matrix :\n",conf)
print("Accuracy score(Doğruluk değeri):\n",accuracy_score(encoded_v, ypred1))
print("\nClassification Report:\n",classification_report(encoded_v, ypred1))
print("MSE:",mean_squared_error(encoded_v, ypred1))

In [ ]:
# Decision Tree Classifier
print("Decision Tree Classifier")
clf = DecisionTreeClassifier()
clf.fit(X_train, Y_train)
y_true5 , y_pred5=y_test,clf.predict(X_test)
print("\nTahmin değerleri: ",y_pred5)
plt.scatter(y_true5, y_pred5,c='orange')
plt.scatter(y_true5, y_test,c='purple')
plt.xlabel("True Values")
plt.ylabel("Predictions")

In [ ]:
#predictions multiclass olduğundan y_validation da multiclassa dönüştürüldü
encoded_v4 = lab_enc.fit_transform(y_true5)
utils.multiclass.type_of_target(y_true5.astype('int'))
ypred5= lab_enc.fit_transform(y_pred5)
utils.multiclass.type_of_target(ypred5.astype('int'))
conf=confusion_matrix(encoded_v4, ypred5)
print("\nConfusion matrix :\n",conf)
print("Accuracy score(Doğruluk değeri):\n",accuracy_score(encoded_v4, ypred5))
print("\nClassification Report:\n",classification_report(encoded_v4, ypred5))
print("MSE:",mean_squared_error(encoded_v4, ypred5))

In [ ]:
# GaussianNB
print("GaussianNB")
clf = GaussianNB()
clf.fit(X_train, Y_train)
y_true3 , y_pred3=y_test,clf.predict(X_test)
print("\nTahmin değerleri: ",y_pred3)
plt.scatter(y_true3, y_pred3,c='pink')
plt.scatter(y_true3, y_test,c='orange')
plt.xlabel("True Values")
plt.ylabel("Predictions")

In [ ]:
# GaussianNB
#predictions multiclass olduğundan y_validation da multiclassa dönüştürüldü
encoded_v2 = lab_enc.fit_transform(y_true3)
utils.multiclass.type_of_target(y_true3.astype('int'))
ypred3= lab_enc.fit_transform(y_pred3)
utils.multiclass.type_of_target(ypred3.astype('int'))
conf=confusion_matrix(encoded_v2, ypred3)
print("\nConfusion matrix :\n",conf)
print("Accuracy score(Doğruluk değeri):\n",accuracy_score(encoded_v2, ypred3))
print("\nClassification Report:\n",classification_report(encoded_v2, ypred3))
print("MSE:",mean_squared_error(encoded_v2, ypred3))

In [ ]:
# SVR(Support Vector Regressions)
print("SVR(Support Vector Regressions)")
clf = svm.SVR(gamma="auto")
# modelimizi eğitim verilerimiz ve buna karşılık gelen Y_train(target ) değerleri ile eğitildi.
clf.fit(X_train, Y_train)
# test değerlerimize karşılık gelecek olan tahmin değerleri oluşturuldu.
y_true2 , y_pred2 =y_test,clf.predict(X_test)
print("\nTahmin değerleri: ",y_pred2)
plt.scatter(y_true2, y_pred2,c='pink')
plt.scatter(y_true2, y_test,c='red')
plt.xlabel("True Values")
plt.ylabel("Predictions")

In [ ]:
#SVR
#predictions multiclass olduğundan y_validation da multiclassa dönüştürüldü
encoded_v1 = lab_enc.fit_transform(y_true2)
utils.multiclass.type_of_target(y_true2.astype('int'))
ypred2= lab_enc.fit_transform(y_pred2)
utils.multiclass.type_of_target(ypred2.astype('int'))
conf=confusion_matrix(encoded_v1, ypred2)
print("\nConfusion matrix :\n",conf)
print("Accuracy score(Doğruluk değeri):\n",accuracy_score(encoded_v1, ypred2))
print("\nClassification Report:\n",classification_report(encoded_v1, ypred2))
print("MSE:",mean_squared_error(encoded_v1, ypred2))

In [ ]:
# KNeighborsClassifier
print("KNeighbors Classifier")
clf = KNeighborsClassifier()
clf.fit(X_train, Y_train)
y_true7 , y_pred7=y_test,clf.predict(X_test)
print("\nTahmin değerleri: ",y_pred7)
plt.scatter(y_true7, y_pred7,c='green')
plt.scatter(y_true7, y_test,c='yellow')
plt.xlabel("True Values")
plt.ylabel("Predictions")

In [ ]:
#predictions multiclass olduğundan y_validation da multiclassa dönüştürüldü
encoded_v6 = lab_enc.fit_transform(y_true7)
utils.multiclass.type_of_target(y_true7.astype('int'))
ypred7= lab_enc.fit_transform(y_pred7)
utils.multiclass.type_of_target(ypred7.astype('int'))
conf=confusion_matrix(encoded_v6, ypred7)
print("\nConfusion matrix :\n",conf)
print("Accuracy score(Doğruluk değeri):\n",accuracy_score(encoded_v6, ypred7))
print("\nClassification Report:\n",classification_report(encoded_v6, ypred7))
print("MSE:",mean_squared_error(encoded_v6, ypred7))

In [ ]:
# Linear Discriminant Analysis
print("Linear Discriminant Analysis")
clf = LinearDiscriminantAnalysis()
clf.fit(X_train, Y_train)
y_true8 , y_pred8=y_test,clf.predict(X_test)
print("\nTahmin değerleri: ",y_pred8)
plt.scatter(y_true8, y_pred8,c='purple')
plt.scatter(y_true8, y_test,c='grey')
plt.xlabel("True Values")
plt.ylabel("Predictions")

In [ ]:
# Linear Discriminant Analysis
#predictions multiclass olduğundan y_validation da multiclassa dönüştürüldü
encoded_v7 = lab_enc.fit_transform(y_true8)
utils.multiclass.type_of_target(y_true8.astype('int'))
ypred8= lab_enc.fit_transform(y_pred8)
utils.multiclass.type_of_target(ypred8.astype('int'))
conf=confusion_matrix(encoded_v7, ypred8)
print("\nConfusion matrix :\n",conf)
sns.heatmap(conf, cmap="Blues")


print("Accuracy score(Doğruluk değeri):\n",accuracy_score(encoded_v7, ypred8))
print("\nClassification Report:\n",classification_report(encoded_v7, ypred8))
print("MSE:",mean_squared_error(encoded_v7, ypred8))